# Aula 16 — Limpeza de Dados: Valores Faltantes (completo)

Notebook gerado automaticamente com análise passo-a-passo e exemplos de imputação.


## 1) Importar bibliotecas e carregar dados

Carregamos pandas, numpy e lemos `titanic.csv`.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_PATH = Path('/mnt/data')
df = pd.read_csv(DATA_PATH / 'titanic.csv')
df.head()

## 2) Resumo de valores faltantes

Quantidade e % de valores faltantes por coluna.

In [ ]:
missing_counts = df.isnull().sum().sort_values(ascending=False)
missing_percent = (df.isnull().mean() * 100).round(2).sort_values(ascending=False)
missing_summary = pd.concat([missing_counts, missing_percent], axis=1)
missing_summary.columns = ['missing_count', 'missing_percent']
missing_summary[missing_summary['missing_count']>0]

## 3) Exemplo — Deleção de observações com `Age` faltante

In [ ]:
df_drop_age = df.dropna(subset=['Age'])
rows_removed = df.shape[0] - df_drop_age.shape[0]
rows_removed, df_drop_age.shape

## 4) Substituição Dummy

In [ ]:
df_dummy = df.copy()
if 'Cabin' in df_dummy.columns:
    df_dummy['Cabin'] = df_dummy['Cabin'].fillna('Unknown')
if 'Age' in df_dummy.columns:
    df_dummy['Age_dummy'] = df_dummy['Age'].fillna(-1)
df_dummy[['PassengerId','Name','Age','Cabin']].head(10)

## 5) Substituição por média/mediana/moda

In [ ]:
df_stat = df.copy()
age_mean = df_stat['Age'].mean()
age_median = df_stat['Age'].median()
embarked_mode = df_stat['Embarked'].mode().iloc[0] if 'Embarked' in df_stat.columns and not df_stat['Embarked'].mode().empty else None

df_stat['Age_mean_imputed'] = df_stat['Age'].fillna(age_mean)
df_stat['Age_median_imputed'] = df_stat['Age'].fillna(age_median)
if embarked_mode is not None:
    df_stat['Embarked_mode_imputed'] = df_stat['Embarked'].fillna(embarked_mode)

age_mean, age_median, embarked_mode

## 6) Imputação por regressão (estimativa de Age)

Usamos RandomForestRegressor para prever Age a partir de Pclass, Sex, SibSp, Parch e Fare.

In [ ]:
# Código para Imputação por regressão (RandomForest)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

predictors = [c for c in ['Pclass','Sex','SibSp','Parch','Fare'] if c in df.columns]
df_reg = df[predictors + ['Age']].copy()

if 'Sex' in df_reg.columns:
    le = LabelEncoder()
    df_reg['Sex_enc'] = le.fit_transform(df_reg['Sex'].astype(str))
    df_reg = df_reg.drop(columns=['Sex'])

if 'Fare' in df_reg.columns:
    df_reg['Fare'] = df_reg['Fare'].fillna(df_reg['Fare'].median())

known = df_reg[df_reg['Age'].notnull()]
unknown = df_reg[df_reg['Age'].isnull()]

if not unknown.empty and len(known) >= 10:
    X_train = known.drop(columns=['Age'])
    y_train = known['Age']
    X_pred = unknown.drop(columns=['Age'])
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_pred)
    df_regressed = df.copy()
    df_regressed.loc[df_regressed['Age'].isnull(), 'Age_regression_imputed'] = preds
    df_regressed['Age_regression_filled'] = df_regressed['Age']
    df_regressed.loc[df_regressed['Age_regression_filled'].isnull(), 'Age_regression_filled'] = df_regressed.loc[df_regressed['Age_regression_filled'].isnull(), 'Age_regression_imputed']
    df_regressed[['PassengerId','Age','Age_regression_imputed','Age_regression_filled']].head(10)
else:
    'Insufficient data for regression imputation or no missing Age'

## 7) Imputação KNN (dados numéricos)

Exemplo com KNNImputer para preencher valores numéricos com base em vizinhos.

In [ ]:
from sklearn.impute import KNNImputer
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
knn_df = df[numeric_cols].copy()
knn_imputer = KNNImputer(n_neighbors=5)
knn_filled = knn_imputer.fit_transform(knn_df)
knn_filled_df = pd.DataFrame(knn_filled, columns=numeric_cols, index=knn_df.index)
knn_filled_df[['Age','Fare']].head(10)

## 8) Salvar dataset tratado (exemplo final)

Estratégia final: Embarked moda, Cabin Unknown, Age pela imputação por regressão quando disponível, senão pela mediana.

In [ ]:
df_final = df.copy()
if 'Embarked' in df_final.columns:
    if not df_final['Embarked'].mode().empty:
        df_final['Embarked'] = df_final['Embarked'].fillna(df_final['Embarked'].mode().iloc[0])
if 'Cabin' in df_final.columns:
    df_final['Cabin'] = df_final['Cabin'].fillna('Unknown')
if 'Age_regression_filled' in globals():
    try:
        df_final['Age'] = df_regressed['Age_regression_filled']
    except Exception:
        df_final['Age'] = df_final['Age'].fillna(df_final['Age'].median())
else:
    if 'Age' in df_final.columns:
        df_final['Age'] = df_final['Age'].fillna(df_final['Age'].median())

# salvar
from pathlib import Path
DATA_PATH = Path('/mnt/data')
df_final.to_csv(DATA_PATH / 'titanic_cleaned.csv', index=False)
'dataset salvo'

## 9) Conclusão e recomendações

- Deleção quando aceitável perder linhas.
- Dummy para sinalizar ausência.
- Média/mediana/moda para imputações simples.
- Modelos (regressão/classificação), KNN ou IterativeImputer para imputações mais informadas.

Os arquivos gerados foram salvos em `/mnt/data`: 
- `completed_Aula16_Limpeza_Valores_Faltantes.ipynb` (este notebook)
- `titanic_cleaned.csv` (dataset tratado)